**Fin 585R**  
**Diether**  
**Problem Set**  
**Momentum Portfolios**  

**Overview**

In this problem set you reproduce your second seminal empirical result in academic finance. Specifically, you reproduce and extend (the original sample was about 1963 to 1990) **the momentum effect** of Jegadeesh and Titman (1993) (see "Returns to Buying Winners and Selling Losers: Implications for Stock Market Efficiency"). This empirical result spawned a huge literature in academic finance, and has been a critical core strategy for quant hedge funds (and others) for the last 30 years. You will find out in the next couple of weeks that models like the CAPM can't explain this portfolio return pattern at all. 

Momentum portfolios are formed based on past returns. Specifically, momentum portfolios are most commonly formed based on the cumulative return from months $t-12$ to $t-2$ (you should use this past return window for your portfolios):

$$
r_{i,t-12:t-2} \approx \sum_{x=2}^{12} \log(1+r_{i,t-x})
$$

Note, it's common practice to cumulate (or compound) the returns using the log approximation (as above). You certainly can do the following if you want (well, not for this problem set ... use log returns for the problem set):

$$
r_{i,t-12:t-2} = \left[ \prod_{x=2}^{12} \bigl(1+r_{i,t-x} \bigr) \right]  - 1
$$

The log approximation is traditionally used in this situation because it's less computational intensive. 

The data for this problem set are monthly observations for all stocks on the NYSE, AMEX, and Nasdaq from July of 1962 to  September of 2022. You can download the data directly using the following link: [the data](https://diether.org/prephd/06-mstk_62-22.csv). There is also a link on *Learning Suite*. The data contain the following variables that you will need for the assignment (it also contains som additional variables):

|Variable | Description                                              |
|---------|----------------------------------------------------------|
|permno   | stock identifier                                         |
|caldt    | calendar date                                            |
|ticker   | ticker symbol                                            |
|prc      | stock price (not lagged, contemporaneous with returns)   |
|me       | market equity (not lagged, contemporaneous with returns) |
|ret      | monthly return                                           |
|shr      | shares outstanding in 1000s                              |


**Tasks**

1. Form quintile based equal-weight momentum portfolios and report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). Note, you should exclude low price stocks from your portfolios (price below $5). We will discuss the code for creating the portfolio formation variable in the class before the assignment. <br><br>

2. Compute the average number of stocks that are in each portfolio.<br><br>

3. Add a spread portfolio (long portfolio 4 and short portfolio 0 $\leftarrow$ it's a zero cost L/S portfolio) to your dataframe of equal-weight momentum portfolios and then compute the summary statistics.<br><br>

4. Form quintile based value-weight momentum portfolios and report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). You should once again have five portfolios (note, the only difference between your equal-weight and value-weight portfolios will be the weights). Note, a value weight portfolio is defined as the following ($me$ refers to the marke value of equity): <br><br>
$$
r_{pt} = \sum_{i=1}^{n} \omega_{i,t-1}r_{it} = \sum_{i=1}^{n} \left(\frac{me_{i,t-1}}{\sum_{j=1}^{n} me_{j,t-1}} \right) r_{it}
$$<br><br>
Hint: think about splitting the formula into the following parts delineated by the parentheses:<br><br>
\begin{align*}
r_{pt} &= \left( \frac{1}{\sum_{j=1}^{n} me_{j,t-1}} \right) \left( \sum_{i=1}^{n} me_{i,t-1} r_{it}
          \right)
 \end{align*}<br><br>
And then compute each part as a separate groupby. Finally, just multiple the resulting dataframes together and you will have computed the value-weight portfolio returns. <br><br>

In [1]:
import pandas as pd
import numpy as np

**Background: Using the `rolling` Method**  

Given we are working with log returns, we cumulate the past returns using a sum. Therefore, `rolling().sum()` is the rolling window method that makes the most sense in this context. Let's take a look at how `rolling().sum()` works.

In [2]:
df = pd.DataFrame({'id':['a','b','c','d','e','f','g'],
                   'val':range(1,8)})
df

,id,val
0,a,1
1,b,2
2,c,3
3,d,4
4,e,5
5,f,6
6,g,7


In [3]:
df['rsum'] = df['val'].rolling(3).sum()
df

,id,val,rsum
0,a,1,NaN
1,b,2,NaN
2,c,3,6.0
3,d,4,9.0
4,e,5,12.0
5,f,6,15.0
6,g,7,18.0


Note, the timing of the `rolling().sum` above. For example, the three observations `rolling(3).sum()` for id = 'd' (row index = 3) is 9 (2+3+4). So the sum include the current observations. We'll have to take the into account to compute returns from t-12 to t-2.

In [4]:
df = pd.DataFrame({'id':['a','b','c','d','e','f','g','h'],
                   'g':['1','1','1','1','2','2','2','2'],
                   'val':range(1,9)})
df

,id,g,val
0,a,1,1
1,b,1,2
2,c,1,3
3,d,1,4
4,e,2,5
5,f,2,6
6,g,2,7
7,h,2,8


In [5]:
df.groupby('g')['val'].rolling(2).sum()

g   
1  0     NaN
   1     3.0
   2     5.0
   3     7.0
2  4     NaN
   5    11.0
   6    13.0
   7    15.0
Name: val, dtype: float64

**Using a Rolling Sum and Shift**

+ Can't just add shift at the end.<br><br>

+ That shifts the whole dataframe. Not shifts within groups<br><br>

+ Use two separate groupbys.

In [6]:
# This is wrong!
df.groupby('g')['val'].rolling(2).sum().shift(1)

g   
1  0     NaN
   1     NaN
   2     3.0
   3     5.0
2  4     7.0
   5     NaN
   6    11.0
   7    13.0
Name: val, dtype: float64

In [7]:
# There is an inconsistency when pandas will add a multi-index vs will not add a multi-index
df['roll'] = df.groupby('g')['val'].rolling(2).sum().reset_index(drop=True)
df['rolllag'] = df.groupby('g')['roll'].shift()
df

,id,g,val,roll,rolllag
0,a,1,1,NaN,NaN
1,b,1,2,3.0,NaN
2,c,1,3,5.0,3.0
3,d,1,4,7.0,5.0
4,e,2,5,NaN,NaN
5,f,2,6,11.0,NaN
6,g,2,7,13.0,11.0
7,h,2,8,15.0,13.0


**Momentum portfolio construction**  

1. Data Preparation.<br><br>

2. Create portfolio formation variable.<br><br>

3. Bin the data<br><br>

4. Create the portfolios based on bins and weights scheme.<br><br>

In [8]:
df = pd.read_csv('06-mstk_62-22.csv',parse_dates=['caldt'])
df

,permno,caldt,ticker,prc,me,ret,shr
0,10000,1986-01-31,OMFGA,4.37500,16.1000,NaN,3680.0
1,10000,1986-02-28,OMFGA,3.25000,11.9600,-0.257143,3680.0
2,10000,1986-03-31,OMFGA,4.43750,16.3300,0.365385,3680.0
3,10000,1986-04-30,OMFGA,4.00000,15.1720,-0.098592,3793.0
4,10000,1986-05-30,OMFGA,3.10938,11.7939,-0.222656,3793.0
...,...,...,...,...,...,...,...
3384890,93436,2022-05-31,TSLA,758.26000,785565.0000,-0.129197,1036010.0
3384891,93436,2022-06-30,TSLA,673.42000,701030.0000,-0.111888,1041000.0
3384892,93436,2022-07-29,TSLA,891.45000,931111.0000,0.323765,1044490.0
3384893,93436,2022-08-31,TSLA,275.61000,863616.0000,-0.072489,3133470.0


**Cumulative rolling past returns**

1. Create log returns.<br><br>

2. Create 12 period cumulative log return windows: t-11 to t-0<br><br>

3. Lag/shift two periods<br><br>

In [9]:
df['logret'] = np.log(1 + df['ret'])
# Rolling second parameter is the MINIMUM period to generate the rolling value
df['mom'] = df.groupby('permno')['logret'].rolling(11,11).sum().reset_index(drop=True)
# Reset to assign back to df; rolls 11 periods because we shift 2
df['mom'] = df.groupby('permno')['mom'].shift(2)
df.head(15)
# Not exactly equal to YOY Return; we use t-2 instead of t-1 because it "works better"

,permno,caldt,ticker,prc,me,ret,shr,logret,mom
0,10000,1986-01-31,OMFGA,4.375000,16.10000,NaN,3680.0,NaN,NaN
1,10000,1986-02-28,OMFGA,3.250000,11.96000,-0.257143,3680.0,-0.297252,NaN
2,10000,1986-03-31,OMFGA,4.437500,16.33000,0.365385,3680.0,0.311436,NaN
3,10000,1986-04-30,OMFGA,4.000000,15.17200,-0.098592,3793.0,-0.103797,NaN
4,10000,1986-05-30,OMFGA,3.109380,11.79390,-0.222656,3793.0,-0.251872,NaN
5,10000,1986-06-30,OMFGA,3.093750,11.73460,-0.005025,3793.0,-0.005038,NaN
6,10000,1986-07-31,OMFGA,2.843750,10.78630,-0.080808,3793.0,-0.084260,NaN
7,10000,1986-08-29,OMFGA,1.093750,4.14859,-0.615385,3793.0,-0.955512,NaN
8,10000,1986-09-30,OMFGA,1.031250,3.91153,-0.057143,3793.0,-0.058841,NaN
9,10000,1986-10-31,OMFGA,0.781250,3.00234,-0.242424,3843.0,-0.277631,NaN


In [10]:
df.groupby('permno')['logret'].rolling(11,11).sum().head(14)

permno    
10000   0          NaN
        1          NaN
        2          NaN
        3          NaN
        4          NaN
        5          NaN
        6          NaN
        7          NaN
        8          NaN
        9          NaN
        10         NaN
        11   -2.138282
        12   -2.079441
        13   -2.390877
Name: logret, dtype: float64

**Lag Variables Before Removing Any Observations**

+ Need to remove missing `mom` observations before binning.<br><br>

+ So lag price and market-cap before you do that.<br><br>

+ I'll remove low priced stock at the same time.<br><br>

+ Remember, you must always impose this restriction using lagged price. Otherwise, you will create a look ahead bias in your portfolio formation.<br><br>

In [11]:
df['prclag'] = df.groupby('permno')['prc'].shift()
df['melag'] = df.groupby('permno')['me'].shift(1)

# We test whether the df has a missing value by checking its equality
# query syntax is fun and easy :)
# lag price is really critical; we need to work with data we already possess!
df = df.query("mom == mom and prclag >= 5").reset_index(drop=True)
df.head(10)

,permno,caldt,ticker,prc,me,ret,shr,logret,mom,prclag,melag
0,10001,1987-02-27,GFGC,6.2500,6.19375,-0.074074,991.0,-0.076961,0.196691,6.7500,6.68925
1,10001,1987-03-31,GFGC,6.3750,6.31763,0.036800,991.0,0.036139,0.140121,6.2500,6.19375
2,10001,1987-04-30,GFGC,6.1250,6.06987,-0.039216,991.0,-0.040005,0.038272,6.3750,6.31763
3,10001,1987-05-29,GFGC,5.6875,5.63631,-0.071429,991.0,-0.074108,0.064559,6.1250,6.06987
4,10001,1987-06-30,GFGC,5.8750,5.82212,0.051429,991.0,0.050150,0.034406,5.6875,5.63631
5,10001,1987-07-31,GFGC,6.0000,5.94600,0.021277,991.0,0.021053,-0.026547,5.8750,5.82212
6,10001,1987-08-31,GFGC,6.5000,6.44150,0.083333,991.0,0.080043,0.033860,6.0000,5.94600
7,10001,1987-09-30,GFGC,6.2500,6.20000,-0.022308,992.0,-0.022560,-0.014767,6.5000,6.44150
8,10001,1987-10-30,GFGC,6.3750,6.32400,0.020000,992.0,0.019803,0.068358,6.2500,6.20000
9,10001,1987-11-30,GFGC,6.1875,6.13800,-0.029412,992.0,-0.029853,0.007331,6.3750,6.32400


**Bin the Data/Create Portfolio Breakpoints**

+ For the short selling loan fee portfolios we used cut to create bins.<br><br>

+ Here we want to create bins based on the quintiles of the `mom` variable so we use `qcut`.<br><br>

+ `qcut` also must be done with a groupby; the quintiles will have different breakpoints ever month.<br><br>

+ Specifically we use `groupby` and transform to call `qcut` every month and transform the `mom` variable into bins.<br><br>

+ Use transform when you're mapping a Nx1 variable (mom) into and new Nx1 variable (bins).<br><br>

+ <b>We must regroup every period because security prices change; we need to keep the same buckets value-weighted </b><br><br>

In [12]:
df.groupby('caldt')['mom'].transform(pd.qcut,5)

0            (0.142, 0.286]
1            (0.0255, 0.16]
2           (-0.13, 0.0423]
3           (0.0465, 0.176]
4           (-0.0212, 0.11]
                 ...       
2215018       (0.189, 2.39]
2215019      (0.104, 2.053]
2215020      (0.107, 1.842]
2215021    (-0.0605, 0.052]
2215022     (0.0933, 1.255]
Name: mom, Length: 2215023, dtype: interval

In [13]:
df.groupby('caldt')['mom'].transform(pd.qcut,5,labels=False)

0          3
1          2
2          1
3          2
4          2
          ..
2215018    4
2215019    4
2215020    4
2215021    3
2215022    4
Name: mom, Length: 2215023, dtype: int64

In [14]:
# The 5 parameter is used to determine the number of buckets
# We can also pass a list of percentages instead of an integer of buckets
# Cut breaks into percentiles to determine bins
df['bins'] = df.groupby('caldt')['mom'].transform(pd.qcut,5,labels=False)


**Tasks**

1. Form quintile based equal-weight momentum portfolios and report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). Note, you should exclude low price stocks from your portfolios (price below $5). We will discuss the code for creating the portfolio formation variable in the class before the assignment. <br><br>

In [15]:
from finance_byu.summarize import summary

In [16]:
port = df.groupby(["caldt", "bins"])['ret'].mean() * 100
port = port.unstack(level = "bins")
summary(port).loc[["count","mean", "std", "tstat", "pval"]].round(3)

bins,0,1,2,3,4
count,711.000,711.000,711.000,711.000,711.000
mean,0.408,0.950,1.137,1.306,1.619
std,6.768,5.142,4.687,4.826,6.322
tstat,1.607,4.929,6.466,7.218,6.826
pval,0.108,0.000,0.000,0.000,0.000


2. Compute the average number of stocks that are in each portfolio.<br><br>

In [17]:
# There are a very similar number of stocks in each portfolio
df.groupby(['caldt', 'bins']).count().reset_index().groupby('bins').permno.mean()
"""
First approach seems to not work correctly
"""
df['stocks'] = 1
df.groupby(['bins', 'caldt']).sum()['stocks'].groupby('bins').mean()
#"""
#But this second, far more robust method produces the same values
#"""

bins
0    623.461322
1    622.880450
2    622.873418
3    622.880450
4    623.267229
Name: stocks, dtype: float64

In [18]:
# Verifying that the numbers are correct:
df.permno.nunique()
"""
This seems to suggest that my average number of stocks per bin is incorrect?
"""
df.groupby(['caldt']).sum().stocks.mean() / 5 # But this tells me I'm correct

623.0725738396625

3. Add a spread portfolio (long portfolio 4 and short portfolio 0 $\leftarrow$ it's a zero cost L/S portfolio) to your dataframe of equal-weight momentum portfolios and then compute the summary statistics.<br><br>

In [19]:
port["spread"] = port[4] - port[0]
port.head(3)

bins,0,1,2,3,4,spread
caldt,,,,,,
1963-07-31,-1.695985,-1.160239,-0.576725,-0.963438,-0.385561,1.310424
1963-08-30,2.669885,4.176108,4.073168,4.495509,6.661783,3.991898
1963-09-30,-1.897316,-1.131404,-0.919125,-1.434696,-1.589049,0.308267


In [20]:
summary(port).loc[["count","mean", "std", "tstat", "pval"]].round(3)

bins,0,1,2,3,4,spread
count,711.000,711.000,711.000,711.000,711.000,711.000
mean,0.408,0.950,1.137,1.306,1.619,1.211
std,6.768,5.142,4.687,4.826,6.322,4.553
tstat,1.607,4.929,6.466,7.218,6.826,7.089
pval,0.108,0.000,0.000,0.000,0.000,0.000


4. Form quintile based value-weight momentum portfolios and report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). You should once again have five portfolios (note, the only difference between your equal-weight and value-weight portfolios will be the weights). Note, a value weight portfolio is defined as the following ($me$ refers to the marke value of equity): <br><br>
$$
r_{pt} = \sum_{i=1}^{n} \omega_{i,t-1}r_{it} = \sum_{i=1}^{n} \left(\frac{me_{i,t-1}}{\sum_{j=1}^{n} me_{j,t-1}} \right) r_{it}
$$<br><br>
Hint: think about splitting the formula into the following parts delineated by the parentheses:<br><br>
\begin{align*}
r_{pt} &= \left( \frac{1}{\sum_{j=1}^{n} me_{j,t-1}} \right) \left( \sum_{i=1}^{n} me_{i,t-1} r_{it}
          \right)
 \end{align*}<br><br>
And then compute each part as a separate groupby. Finally, just multiple the resulting dataframes together and you will have computed the value-weight portfolio returns. <br><br>

In [21]:
# Sum the total value of equities by month
sum_equity = df.groupby(['caldt', 'bins']).melag.sum() # A vector of lagged market equities that we require

# Add total equity as a column and generate weights row-wise
# I know this is a pretty slow way to do this but I'm not sure how else to do it...
df['te'] = df.apply(lambda row: sum_equity[row.caldt, row.bins], axis = 1)
df['weight'] = df['melag'] / df['te']
df.head(3)

,permno,caldt,ticker,prc,me,ret,shr,logret,mom,prclag,melag,bins,stocks,te,weight
0,10001,1987-02-27,GFGC,6.250,6.19375,-0.074074,991.0,-0.076961,0.196691,6.750,6.68925,3,1,838101.97592,0.000008
1,10001,1987-03-31,GFGC,6.375,6.31763,0.036800,991.0,0.036139,0.140121,6.250,6.19375,2,1,489175.79421,0.000013
2,10001,1987-04-30,GFGC,6.125,6.06987,-0.039216,991.0,-0.040005,0.038272,6.375,6.31763,1,1,415469.40147,0.000015


In [22]:
# Verify the weights add up to one for an arbitrary date; this implies my alg was likely correct
df.query("caldt == '2022-07-29'").weight.sum()

5.0

In [23]:
# Generate portfolio return; the multiple of the standard return and the associated weight
port_return = df.weight * df.ret

# ... and save it to the dataframe
df['value_weight_return'] = port_return

df.head(3)

,permno,caldt,ticker,prc,me,ret,shr,logret,mom,prclag,melag,bins,stocks,te,weight,value_weight_return
0,10001,1987-02-27,GFGC,6.250,6.19375,-0.074074,991.0,-0.076961,0.196691,6.750,6.68925,3,1,838101.97592,0.000008,-5.912170e-07
1,10001,1987-03-31,GFGC,6.375,6.31763,0.036800,991.0,0.036139,0.140121,6.250,6.19375,2,1,489175.79421,0.000013,4.659470e-07
2,10001,1987-04-30,GFGC,6.125,6.06987,-0.039216,991.0,-0.040005,0.038272,6.375,6.31763,1,1,415469.40147,0.000015,-5.963141e-07


In [24]:
# And finally compute summary statistics over the bins, which have stayed the same
port = df.groupby(["caldt", "bins"])['value_weight_return'].sum() * 100
port = port.unstack(level = "bins")
summary(port).loc[["count","mean", "std", "tstat", "pval"]].round(3)

bins,0,1,2,3,4
count,711.000,711.000,711.000,711.000,711.000
mean,0.427,0.815,0.854,1.018,1.321
std,6.624,4.836,4.345,4.462,5.712
tstat,1.719,4.494,5.242,6.083,6.167
pval,0.086,0.000,0.000,0.000,0.000


In [25]:
port

bins,0,1,2,3,4
caldt,,,,,
1963-07-31,-1.684790,-0.566724,0.611594,0.141664,-0.317750
1963-08-30,5.283128,5.319040,4.752087,4.836893,6.456210
1963-09-30,-3.279339,0.664446,-1.051720,-1.395222,-2.508058
1963-10-31,2.369550,-0.283502,2.815613,1.314008,6.873582
1963-11-29,-0.775725,1.814131,-1.304004,-1.915759,-0.519791
...,...,...,...,...,...
2022-05-31,-4.686609,2.988749,-0.082342,-0.763466,-0.343652
2022-06-30,-11.458108,-10.989660,-7.952733,-6.587350,-7.691200
2022-07-29,11.306234,13.595063,9.043699,10.726340,7.016211


I have no idea is this is right lol